# PCA and k-Nearest Neighbours

Consider the Digits dataset that is a part of the sklearn library. It consists of 1797 64 dimensional vectors with each corresponding to an 8x8 image of a digit. The label also gives the digit id. It is a 10-class classification problem.

Choose a random subset of size 1500 for train and the rest for testing. Run k-Nearest neighbours with k values 1,3,7,15 and 31 and report the training and test accuracy. 

Repeat the above after performing PCA on the data. Use top n-principal components for n=2,4,8,16,32. For each n in the list report the best k-NN test accuracy and the k which achieves that accuracy and the approximation error for this particular value of n.

Repeat the above for a noisy version of the data. i.e. add a random Gaussian noise of mean zero and variance 1 to all the 1797*64 input numbers.

In total, the results should be given in 4 tables in the last textwrite cell:. Summarise your findings in a paragraph.

Table 1: Raw data , k-NN performance. One row for each k.

Table 2: n-component PCA preprocessed data k-NN performance. One row for each n.

Table 3: Raw noised data, k-NN performance. One row for each k.

Table 4: n-component PCA preprocessed noised data k-NN performance. One row for each n.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

In [2]:
# Codewrite cell (Use as you wish)

# Train-test split function
def train_test_split(X, y, size_train, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    size_train = int(size_train)
    train_indices = indices[:size_train]
    test_indices = indices[size_train:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# PCA implementation
def pca(X, n_components):
    mean = np.mean(X, axis=0)
    X_ = X - mean
    cov_mat = np.cov(X_, rowvar=False) 
    # rowvar=False means that each column represents a feature, and each row represents an observation data point
    eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]
    sorted_eigenvectors = sorted_eigenvectors[:, :n_components]
    X_reduced = np.dot(X_, sorted_eigenvectors)
    return X_reduced, sorted_eigenvectors, mean

def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

# KNN implementation
def k_nearest_neighbors(X_train, y_train, X_test, k):
    y_pred = []
    for test_point in X_test:
        distances = []
        for i, train_point in enumerate(X_train):
            distance = euclidean_distance(test_point, train_point)
            distances.append((distance, y_train[i]))
        distances.sort(key=lambda x: x[0])
        k_nearest_labels = [label for _, label in distances[:k]]
        majority_vote = np.argmax(np.bincount(k_nearest_labels))
        y_pred.append(majority_vote)
    return np.array(y_pred)

# Accuracy calculation
def accuracy(y_true, y_pred):
    return (np.sum(y_true == y_pred) / len(y_true))*100

def run_knn(X_train, X_test, y_train, y_test, k_values):
    results = []
    for k in k_values:
        y_train_pred = k_nearest_neighbors(X_train, y_train, X_train, k)
        train_acc = accuracy(y_train, y_train_pred)
        y_test_pred = k_nearest_neighbors(X_train, y_train, X_test, k)
        test_acc = accuracy(y_test, y_test_pred)
        results.append((k, train_acc, test_acc))
    return results

# Loading the dataset
X, y = load_digits().data, load_digits().target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, size_train=1500, random_state=42)

# Define k values
k_values = [1, 3, 7, 15, 31]

# PCA n values
pca_n_values = [2, 4, 8, 16, 32]

# Print result tables
def print_table(header, data):
    print(header)
    for row in data:
        print("  ".join(f"{item:.4f}" if isinstance(item, float) else str(item) for item in row))
    print()


In [3]:
# Do the experiments for filling Tables 1 and 2 here
# Raw data k-NN performance
raw_results = run_knn(X_train, X_test, y_train, y_test, k_values)

# print("Table 1: Raw data k-NN performance")
# print_table(["k", "Train Accuracy", "Test Accuracy"], raw_results)

pca_results = []

for n in pca_n_values:
    X_train_pca, eigenvectors, mean= pca(X_train, n)
    X_test_pca = np.dot(X_test - mean, eigenvectors)
    pca_knn_results = run_knn(X_train_pca, X_test_pca, y_train, y_test, k_values)
    best_k, best_train_acc, best_test_acc = max(pca_knn_results, key=lambda item: item[2])
    approximation_error = np.mean((X_test - np.dot(X_test_pca, eigenvectors.T) - mean) ** 2)
    pca_results.append((n, best_k, best_test_acc, approximation_error))

# print("Table 2: PCA-preprocessed data k-NN performance")
# print_table(["n", "Best k", "Best Test Accuracy", "Approximation Error"], pca_results)


In [4]:
# Do the experiments for filling Tables 3 and 4 here

# Adding Gaussian noise to data
np.random.seed(42)
X_noised = X + np.random.normal(0, 1, X.shape)
X_train_noised, X_test_noised, y_train, y_test = train_test_split(X_noised, y, size_train=1500, random_state=42)

# Noised raw data k-NN performance
noised_raw_results = run_knn(X_train_noised, X_test_noised, y_train, y_test, k_values)

# print("Table 3: Noised raw data k-NN performance")
# print_table(["k", "Train Accuracy", "Test Accuracy"], noised_raw_results)

noised_pca_results = []

for n in pca_n_values:
    X_train_noised_pca, eigenvectors, mean = pca(X_train_noised, n)
    X_test_noised_pca = np.dot(X_test_noised - mean, eigenvectors)
    noised_pca_knn_results = run_knn(X_train_noised_pca, X_test_noised_pca, y_train, y_test, k_values)
    best_k, best_train_acc, best_test_acc = max(noised_pca_knn_results, key=lambda item: item[2])
    approximation_error = np.mean((X_test_noised - np.dot(X_test_noised_pca, eigenvectors.T) - mean) ** 2)
    noised_pca_results.append((n, best_k, best_test_acc, approximation_error))

# print("Table 4: PCA-preprocessed noised data k-NN performance")
# print_table(["n", "Best k", "Best Test Accuracy", "Approximation_error"], noised_pca_results)

# Textwrite cell

**Table 1: Raw data k-NN performance**

k | Train Accuracy | Test Accuracy 
--- | --- | --- 
1 | 100.00% |  98.32% 
3 |  99.27% |   98.65% 
7 | 98.87%  |  97.98% 
15 | 98.33% |  97.31% 
31 | 97.00% |  95.96% 

**Table 2: PCA-preprocessed data k-NN performance**

n | Best k | Best Test Accuracy | Approxiamation error
--- | --- | --- | ---
2 | 15 |  64.65% | 13.43
4 |  7 |   87.54% | 9.64
8 | 3  |  96.30% | 6.22
16 | 3 |  98.99% | 2.84
32 | 3 |  98.65% | 0.65

**Table 3: Noised raw data k-NN performance**

k | Train Accuracy | Test Accuracy 
--- | --- | --- 
1 | 100.00% |  98.32% 
3 |  99.20% |   98.65% 
7 | 98.73%  |  98.32% 
15 | 98.26% |  97.64% 
31 | 97.13% |  95.96% 

**Table 4: PCA-preprocessed data k-NN performance**

n | Best k | Best Test Accuracy | Approxiamation error
--- | --- | --- | ---
2 | 15 |  65.32% | 14.34
4 |  15 |   86.19% | 10.52
8 | 3  |  96.30% | 7.05
16 | 3 |  98.32% | 3.55
32 | 1 |  98.32% | 1.18

**Summary of Findings**

#### 1. Raw Data k-NN Performance
- The results show that the k-NN algorithm performs well on the raw data.
- The training accuracy decreases slightly as k increases.
- The test accuracy is consistently high across different k values, with the highest accuracy at k=3 (98.65%).

#### 2. PCA-Preprocessed Data k-NN Performance
- The performance improves significantly as the number of principal components increases.
- For small numbers of components (e.g., n=2), the accuracy is much lower (64.65%), indicating that the data is not well-represented.
- The accuracy improves as more components are used, reaching near raw data performance at n=16 and n=32.

#### 3. Noised Raw Data k-NN Performance
- Test accuracy is consistent across different k values, with the highest accuracy again at k=3 (98.65%).

#### 4. PCA-Preprocessed Noised Data k-NN Performance
- The accuracy for the noised PCA-preprocessed data follows a similar trend to the clean PCA data.
- For small numbers of components (e.g., n=2), the accuracy is low (65.32%).
- As the number of components increases, the accuracy improves, nearing the performance of the noised raw data at n=16 and n=32.

### Explanation for Observed Values
1. **Raw Data Performance**: The high performance on raw data is likely due to the high-dimensional feature space that k-NN can effectively utilize.

2. **PCA-Preprocessed Data Performance**:
    - With fewer components, significant information loss occurs, leading to lower accuracy.
    - As the number of components increases, more information is retained, improving accuracy.
    - With n=16 and n=32 components, most of the variance in the data is captured, resulting in accuracy close to the raw data.

3. **Effect of Noise**:
    - The noise introduces some distortion, but k-NN is robust to a degree of noise.
    - The performance on noised raw data remains high because the noise level is not enough to significantly disrupt the high-dimensional structure that k-NN relies on.
    - PCA helps mitigate noise by reducing dimensionality, which can help focus on the most relevant features, but the initial reduction in dimensions must be carefully chosen to avoid information loss.

4. **Approximation Error**:
    - The error is calculated as the difference between the test accuracy on raw data and PCA-preprocessed data.
    - Higher approximation error at lower n values indicates significant information loss.
    - Lower approximation error at higher n values indicates that PCA is capturing more of the data's variance, preserving performance.
